<a href="https://colab.research.google.com/github/ahatasham5/Breast_Cancer_Prognosis/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <div style="color:yellow;display:inline-block;border-radius:5px;background-color:#007BA7;font-family:Nexa;overflow:hidden"><p style="padding:15px;color:yellow;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b> </b> Import Modules</p></div>

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [ ]:
import torch
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # For binary classification

# Check if a GPU is available and move the model to GPU if it is
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 72.1MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Dataset/

/content/drive/MyDrive/Dataset


In [ ]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_weights.pth


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])

# Load your dataset
dataset = datasets.ImageFolder("Biglycan breast cancer dataset", transform=transform)

# Splitting dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_weights.pth


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [ ]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_weights.pth


In [ ]:
import torch

# Define the checkpoint file path
checkpoint_path = 'model_checkpoint.pth'

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

Epoch 1, Loss: 0.7047864728503757
Epoch 2, Loss: 0.6176774435573154


KeyboardInterrupt: 

In [ ]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_checkpoint.pth   model_weights.pth


In [ ]:
import torch

# Define the checkpoint file path
checkpoint_path = 'model_checkpoint.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)
start_epoch = checkpoint['epoch'] + 1
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']

print(f"Resuming training from epoch {start_epoch}, with loss: {loss}")

num_epochs = 100
for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

Resuming training from epoch 2, with loss: 0.6176774435573154
Epoch 3, Loss: 0.5605335997210609
Epoch 4, Loss: 0.4961458577050103
Epoch 5, Loss: 0.469496660762363
Epoch 6, Loss: 0.4398384624057346
Epoch 7, Loss: 0.40820057193438214
Epoch 8, Loss: 0.3925192753473918
Epoch 9, Loss: 0.3748172190454271
Epoch 10, Loss: 0.3886091444227431
Epoch 11, Loss: 0.36517011125882465
Epoch 12, Loss: 0.3168796118762758
Epoch 13, Loss: 0.3232196122407913
Epoch 14, Loss: 0.30815551347202724
Epoch 15, Loss: 0.3359830743736691
Epoch 16, Loss: 0.33848564989036983
Epoch 17, Loss: 0.32079076435830856
Epoch 18, Loss: 0.28307667043473983
Epoch 19, Loss: 0.28098246455192566
Epoch 20, Loss: 0.29133709106180405
Epoch 21, Loss: 0.2965857833623886
Epoch 22, Loss: 0.2717053559091356
Epoch 23, Loss: 0.25783400071991813
Epoch 24, Loss: 0.2649542639652888
Epoch 25, Loss: 0.2641727858119541
Epoch 26, Loss: 0.2515537424219979
Epoch 27, Loss: 0.24705953233771855
Epoch 28, Loss: 0.23805179860856798
Epoch 29, Loss: 0.2388336

KeyboardInterrupt: 

In [ ]:
import torch

# Load the saved model weights
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [ ]:
import torch

# Load the checkpoint
checkpoint = torch.load('model_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# Optionally, you can also load the optimizer state and other saved data
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

In [ ]:
model.eval()  # Set model to evaluation mode
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Calculate and print the evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(all_targets, np.array(all_preds).flatten()))

print("\nClassification Report:")
print(classification_report(all_targets, np.array(all_preds).flatten(), target_names=['Healthy', 'Cancerous']))


Confusion Matrix:
[[38  1]
 [ 7 22]]

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.84      0.97      0.90        39
   Cancerous       0.96      0.76      0.85        29

    accuracy                           0.88        68
   macro avg       0.90      0.87      0.88        68
weighted avg       0.89      0.88      0.88        68



In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        # Apply sigmoid since we used BCEWithLogitsLoss
        predicted_probs = torch.sigmoid(outputs)

        # Convert probabilities to predicted class (0 or 1)
        predicted = predicted_probs > 0.5

        total += labels.size(0)
        correct += (predicted.flatten() == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 88.24%


# <div style="color:yellow;display:inline-block;border-radius:5px;background-color:#007BA7;font-family:Nexa;overflow:hidden"><p style="padding:15px;color:yellow;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b> </b>Exploratory Data Analysis (EDA)📊</p></div>

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>1. What is the primary objective of using deep learning in breast cancer analysis?</b></font>

**Answer: The primary objective is to enhance breast cancer analysis through deep learning.**

**Explanation: Deep learning aims to improve various aspects, including diagnosis, prognosis, treatment planning, and research. It leverages complex neural networks to extract patterns and features from medical images, leading to more accurate and efficient analysis.**


In [ ]:
# Create a sunburst chart to visualize the primary objectives of using deep learning in breast cancer analysis
fig = go.Figure(go.Sunburst(
    labels=["Diagnosis", "Prognosis", "Treatment Planning", "Research"],
    parents=["", "", "", ""],
))
fig.update_layout(title="Primary Objectives of Deep Learning in Breast Cancer Analysis")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>2. How can convolutional neural networks (CNNs) be beneficial for image-based cancer detection?</b></font>

**Answer: CNNs are beneficial for image-based cancer detection due to their hierarchical feature extraction.**
    
**Explanation: They can automatically learn and extract relevant features at different levels of abstraction, enabling the model to understand complex patterns in medical images. This hierarchical representation enhances the model's ability to detect subtle patterns indicative of cancer.**


In [ ]:
# Create a 3D surface plot to illustrate the hierarchical feature extraction of CNNs
fig = go.Figure(data=[go.Surface(z=[[1, 1, 1, 1], [1, 2, 2, 1], [1, 2, 2, 1], [1, 1, 1, 1]])])
fig.update_layout(title="Hierarchical Feature Extraction in CNNs")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>3. What role does the "Cancer" column play in the dataset, and how is it crucial for model training?</b></font>

**Answer: The 'Cancer' column indicates the class labels (1 for cancer, 0 for healthy) in the dataset.**

**Explanation: It is crucial for model training as it serves as the ground truth for supervised learning. The model learns to map input images to these labels during training, enabling it to make predictions on new, unseen data.**


In [ ]:
# Create a pie chart to visualize the distribution of "Cancer" classes in the dataset
fig = px.pie(df, names='Cancer', title='Distribution of Cancer Classes')
fig.show()


<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>4. Describe the structure of a typical convolutional neural network used for image classification in cancer detection.</b></font>

**Answer: A typical CNN structure for image classification consists of convolutional layers followed by fully connected layers.**

**Explanation: Convolutional layers extract spatial hierarchies of features, and fully connected layers combine these features for final classification. This architecture allows the model to learn hierarchical representations of image content.**


In [ ]:
# Create a diagram to illustrate the structure of a typical CNN
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=["Input", "Convolutional Layers", "Fully Connected Layers", "Output"],
    ),
    link=dict(
        source=[0, 0, 1, 1, 2],
        target=[1, 2, 2, 3, 3],
        value=[1, 2, 2, 1, 1],
    ),
))
fig.update_layout(title="Typical CNN Structure for Image Classification")
fig.show()


<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>5. Explain the significance of the file paths ("/CANCER/252214-11.png") in the dataset for deep learning.</b></font>

**Answer: File paths are crucial for locating and loading individual images during deep learning.**

**Explanation: They serve as unique identifiers for images in the dataset, helping the model associate each image with its corresponding label. Proper file path handling is essential for effective data loading and preprocessing.**


In [ ]:
ls


sample_data/


In [ ]:
%cd ..

/content/drive/MyDrive/Dataset


In [ ]:
ls

sample_data/


In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [ ]:
import torch
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # For binary classification

# Check if a GPU is available and move the model to GPU if it is
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 128MB/s]


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])

# Load your dataset
dataset = datasets.ImageFolder("Biglycan breast cancer dataset/", transform=transform)

# Splitting dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
ls


sample_data/


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
import torch

# Define the checkpoint file path
checkpoint_path = '/model_checkpoint.pth'

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

NameError: name 'model' is not defined

In [ ]:
ls

sample_data/


In [ ]:
num_epochs = 100  # Define the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.6585823363727994
Epoch 2, Loss: 0.5549289782842001
Epoch 3, Loss: 0.49389655391375226
Epoch 4, Loss: 0.4982609450817108
Epoch 5, Loss: 0.41697002450625104
Epoch 6, Loss: 0.4228925373819139
Epoch 7, Loss: 0.41234508487913346
Epoch 8, Loss: 0.3698950211207072
Epoch 9, Loss: 0.38001003199153477
Epoch 10, Loss: 0.3632267547978295
Epoch 11, Loss: 0.3563547233740489
Epoch 12, Loss: 0.36078233851326835
Epoch 13, Loss: 0.3739441649781333
Epoch 14, Loss: 0.34654802415106034
Epoch 15, Loss: 0.31369953023062813
Epoch 16, Loss: 0.28713630470964646
Epoch 17, Loss: 0.2856965313355128
Epoch 18, Loss: 0.28133418493800694
Epoch 19, Loss: 0.27583395110236275
Epoch 20, Loss: 0.27449676560031044
Epoch 21, Loss: 0.267212364408705
Epoch 22, Loss: 0.27017222179306877
Epoch 23, Loss: 0.2643633037805557
Epoch 24, Loss: 0.2408582849635018
Epoch 25, Loss: 0.25632908940315247
Epoch 26, Loss: 0.2435330632660124
Epoch 27, Loss: 0.2366712854968177
Epoch 28, Loss: 0.26844314899709487


KeyboardInterrupt: 

In [ ]:
# After training for some epochs
torch.save(model.state_dict(), 'model_weights.pth')
# or
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    # Any other relevant data
}, 'checkpoint.pth')

In [ ]:
ls


'Biglycan breast cancer dataset'/   checkpoint.pth   model_weights.pth


In [ ]:
# Load model weights
model.load_state_dict(torch.load('model_weights.pth'))

# Load checkpoint
checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# Load any other relevant data

In [ ]:
ls


'Biglycan breast cancer dataset'/


In [ ]:
import torch

# Define the checkpoint file path
checkpoint_path = 'checkpoint.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)
start_epoch = checkpoint['epoch'] + 1
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']

print(f"Resuming training from epoch {start_epoch}, with loss: {loss}")

num_epochs = 100
for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

KeyError: 'loss'

In [ ]:
# Display a sample image along with its file path
sample_image_path = df.iloc[0]['imagem']
sample_image = Image.open("/kaggle/input/biglycan-breast-cancer/Biglycan breast cancer dataset" + sample_image_path)

fig = px.imshow(sample_image)
fig.update_layout(title="Sample Image with File Path: " + sample_image_path)
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/biglycan-breast-cancer/Biglycan breast cancer dataset/CANCER/252214-11.png'

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>6. What preprocessing steps might be necessary for preparing image data for deep learning models in cancer analysis?</b></font>

**Answer: Common preprocessing steps include resizing and normalization of images.**

**Explanation: Resizing ensures consistent input dimensions for the model, and normalization scales pixel values to a standard range. Additional steps may include data augmentation, cropping, and histogram equalization depending on the dataset and model requirements.**


In [ ]:
from PIL import Image
import numpy as np
import plotly.express as px

def preprocess_image(image_path, target_size=(224, 224)):
    # Open the image
    img = Image.open(image_path)

    # Resize the image
    img = img.resize(target_size)

    # Convert the image to a numpy array
    img_array = np.array(img)

    # Normalize the pixel values to be between 0 and 1
    img_array = img_array / 255.0

    return img_array

# Assuming sample_image is the file name or part of the path
sample_image = "252214-11.png"  # Replace with the actual file name
sample_image_path = "/kaggle/input/biglycan-breast-cancer/Biglycan breast cancer dataset/CANCER/" + sample_image
preprocessed_image = preprocess_image(sample_image_path)

# Visualize the preprocessed image
fig = px.imshow(preprocessed_image)
fig.update_layout(title="Preprocessed Image Example")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>7. How could transfer learning be employed to enhance the performance of a deep learning model on this breast cancer dataset?</b></font>

**Answer: Transfer learning can enhance performance by leveraging knowledge from pre-trained models.**

**Explanation: Feature extraction involves using pre-trained layers as feature extractors, while fine-tuning adapts specific layers to the new task. This utilizes knowledge gained from a different but related task, improving model performance, especially with limited data.**



In [ ]:
# Illustrate the concept of transfer learning using a bar chart
fig = px.bar(x=['Feature Extraction', 'Fine-Tuning'], y=[0.7, 0.9], labels={'x': 'Transfer Learning', 'y': 'Accuracy'})
fig.update_layout(title="Transfer Learning Strategies")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>8. Discuss the potential challenges associated with classifying breast cancer images using deep learning.</b></font>

**Answer: Challenges in classifying breast cancer images include data imbalance, limited data, heterogeneity, noise, and interpretability.**

**Explanation: Addressing these challenges requires careful consideration of data preprocessing, model architecture, and evaluation metrics. Strategies such as data augmentation, transfer learning, and attention mechanisms can mitigate some of these challenges.**


In [ ]:
# Visualize a word cloud to represent challenges
challenges = ["Data Imbalance", "Limited Data", "Heterogeneity", "Noise", "Interpretability"]
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(Counter(challenges))

fig = px.imshow(wordcloud)
fig.update_layout(title="Challenges in Classifying Breast Cancer Images")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>9. What is the impact of imbalanced classes (e.g., more cancer cases than healthy cases) on deep learning model training?</b></font>

**Answer: Imbalanced classes can impact model training, particularly when one class dominates the dataset.**

**Explanation: The model may become biased towards the majority class, leading to suboptimal performance on the minority class. Techniques such as class weighting, oversampling, or using specific evaluation metrics can help address this imbalance.**


In [ ]:
# Visualize a bar chart showing the class distribution
class_distribution = df['Cancer'].value_counts()

fig = px.bar(x=class_distribution.index, y=class_distribution.values, labels={'x': 'Class', 'y': 'Count'})
fig.update_layout(title="Class Distribution in the Dataset")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>10. How might hyperparameter tuning be performed to optimize the deep learning model's performance on this dataset?</b></font>

**Answer: Hyperparameter tuning involves systematically adjusting model parameters to optimize performance.**

**Explanation: Techniques like grid search or random search can be employed to explore different hyperparameter combinations. The impact of hyperparameter values on model performance is typically visualized to identify the optimal configuration.**


In [ ]:
# Visualize a line plot showing the impact of hyperparameter values on model performance
hyperparameter_values = [50, 100, 150, 200]
accuracy_scores = [0.75, 0.78, 0.82, 0.81]

fig = px.line(x=hyperparameter_values, y=accuracy_scores, labels={'x': 'Hyperparameter Value', 'y': 'Accuracy'})
fig.update_layout(title="Hyperparameter Tuning Impact on Model Performance")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>11. What metrics could be used to evaluate the performance of the deep learning model in cancer detection?</b></font>

**Answer: Multiple metrics can be used to evaluate the performance of a deep learning model in cancer detection.**

**Explanation: Common metrics include accuracy, precision, recall, F1 score, and ROC-AUC. These metrics provide a comprehensive understanding of the model's capabilities in terms of true positives, false positives, true negatives, and false negatives.**


In [ ]:
import plotly.graph_objects as go

# Visualize a radar chart showing multiple evaluation metrics
metrics = ["Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"]
metric_values = [0.85, 0.78, 0.82, 0.80, 0.90]

# Duplicate the first metric at the end to close the line
metrics += [metrics[0]]
metric_values += [metric_values[0]]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=metric_values,
    theta=metrics,
    fill='toself'
))

fig.update_layout(title="Model Evaluation Metrics for Cancer Detection")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>12. Explain the concept of feature visualization in the context of deep learning for breast cancer analysis.</b></font>

**Answer: "Feature visualization involves visualizing activated regions within an image to understand what the model focuses on.**

**Explanation: Activation maps, obtained from intermediate layers of the model, highlight regions relevant to the model's decision. This aids interpretability and can provide insights into the features important for cancer detection.**


In [ ]:
# Visualize a heatmap showing activated regions in an image
activated_regions = np.random.rand(10, 10)  # Replace with actual activation map
fig = px.imshow(activated_regions, color_continuous_scale='Viridis')
fig.update_layout(title="Feature Visualization - Activated Regions")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>13. How could attention mechanisms be applied to highlight regions of interest in breast cancer images?</b></font>

**Answer: Attention mechanisms can be applied to highlight specific regions of interest in breast cancer images.**

**Explanation: These mechanisms assign different weights to pixels, emphasizing areas crucial for classification. By visualizing the attention map, one can identify regions where the model focuses its attention, aiding in model interpretability.**


In [ ]:
# Visualize an attention heatmap applied to an image
attention_map = np.random.rand(10, 10)  # Replace with actual attention map
fig = px.imshow(attention_map, color_continuous_scale='Reds')
fig.update_layout(title="Attention Mechanism - Highlighted Regions")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>14. Discuss the ethical considerations involved in deploying deep learning models for medical diagnoses, specifically in breast cancer detection.</b></font>

**Answer: Deploying deep learning models for medical diagnoses raises important ethical considerations.**

**Explanation: These include concerns about bias, privacy of patient data, transparency in model decision-making, accountability for errors, and ensuring equitable access to healthcare solutions. Addressing these considerations is vital for responsible and fair deployment.**



In [ ]:
# Visualize a word cloud representing ethical considerations
ethical_considerations = ["Bias", "Privacy", "Transparency", "Accountability", "Equity"]
wordcloud_ethical = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(Counter(ethical_considerations))

fig = px.imshow(wordcloud_ethical)
fig.update_layout(title="Ethical Considerations in Deploying Deep Learning Models")
fig.show()


<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>15. Explore the potential impact of noise or artifacts in the images on the accuracy of the deep learning model.</b></font>

**Answer: Noise or artifacts in images can negatively impact the accuracy of a deep learning model.**

**Explanation: These disturbances may introduce misleading patterns or reduce the model's ability to recognize relevant features. Robust preprocessing and model training techniques are essential to mitigate the impact of noise and artifacts.**


In [ ]:
import numpy as np

def add_noise_to_image(image, noise_level=30):
    """
    Add Gaussian noise to the input image.

    Parameters:
        image (PIL.Image.Image): The input image.
        noise_level (int): The standard deviation of the Gaussian noise.

    Returns:
        PIL.Image.Image: The image with added noise.
    """
    np_image = np.array(image)
    noise = np.random.normal(scale=noise_level, size=np_image.shape).astype(np.uint8)
    noisy_image = np.clip(np_image + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)

In [ ]:
from PIL import Image
from plotly.subplots import make_subplots
import plotly.express as px

# Assuming sample_image is the filename of an image in your dataset
sample_image = "13208 (12) (1).png"
sample_image_path = "/kaggle/input/biglycan-breast-cancer/Biglycan breast cancer dataset/CANCER/" + sample_image

# Visualize a comparison between a clean image and an image with simulated noise/artifacts
clean_image = Image.open(sample_image_path)
noisy_image = add_noise_to_image(clean_image)

fig = make_subplots(rows=1, cols=2, subplot_titles=["Clean Image", "Noisy Image"])
fig.add_trace(px.imshow(clean_image).data[0], row=1, col=1)
fig.add_trace(px.imshow(noisy_image).data[0], row=1, col=2)

fig.update_layout(title="Impact of Noise/Artifacts on Model Accuracy")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>16. How can interpretability and explainability be addressed in deep learning models for breast cancer detection?</b></font>

**Answer: Addressing interpretability and explainability involves designing models that provide insights into their decision-making process.**

**Explanation: Interpretable models, such as decision trees or attention-based models, offer transparency, making it easier to understand and trust the model's predictions. Balancing model complexity with interpretability is crucial for medical applications.**

In [ ]:
# Visualize a comparison of a black-box model and an interpretable model
fig = make_subplots(rows=1, cols=2, subplot_titles=["Black-Box Model", "Interpretable Model"])
fig.add_trace(px.scatter(x=np.random.rand(100), y=np.random.rand(100), opacity=0.5).data[0], row=1, col=1)
fig.add_trace(px.scatter(x=np.random.rand(100), y=np.random.rand(100), opacity=0.5).data[0], row=1, col=2)

fig.update_layout(title="Interpretability in Model Design")
fig.show()

<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>17. Investigate the importance of fine-tuning hyperparameters based on the characteristics of the medical imaging dataset.</b></font>

**Answer: Fine-tuning hyperparameters is crucial for adapting the model to the characteristics of the medical imaging dataset.**

**Explanation: By carefully adjusting hyperparameters based on the dataset's unique features, one can optimize the model's performance, leading to improved accuracy and robustness in breast cancer detection.**


In [ ]:
# Visualize a comparison of model performance with and without hyperparameter fine-tuning
fig = px.bar(x=['Without Fine-Tuning', 'With Fine-Tuning'], y=[0.75, 0.85], labels={'x': 'Hyperparameter Fine-Tuning', 'y': 'Accuracy'})
fig.update_layout(title="Impact of Hyperparameter Fine-Tuning on Model Performance")
fig.show()


<div style="border-radius: 10px; border: 2px solid #FFD700; padding: 15px; background-color:#FFFACD; font-size: 100%; text-align: left;">
    
<font size="+1" color="#0000FF"><b>18. What are some potential future directions or advancements in deep learning for breast cancer research and diagnosis?</b></font>

**Answer: Future directions in deep learning for breast cancer research and diagnosis encompass several exciting avenues.**

**Explanation: These include advancements in early detection methods, personalized medicine approaches, the integration of explainable AI for clinical adoption, exploration of multi-modal data fusion, and fostering large-scale collaborative efforts to enhance research impact.**


In [ ]:
# Visualize a word cloud representing potential future directions
future_directions = ["Early Detection", "Personalized Medicine", "Explainable AI", "Multi-Modal Fusion", "Large-Scale Collaboration"]
wordcloud_future = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(Counter(future_directions))

fig = px.imshow(wordcloud_future)
fig.update_layout(title="Potential Future Directions in Deep Learning for Breast Cancer")
fig.show()

<div class="alert alert-block alert-info">"Your positive feedback and upvote mean a lot! It motivates me to create more valuable content and helps others discover it too. Let's build a thriving community of knowledge-sharing. Thank you for your support! 😊"</div>